In [2]:
import pandas as pd
import altair as alt

data = pd.read_csv("./Databases/compiled_db.csv")
data.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,...,Publisher,Year Released,Month Released,Game Type,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Genre 6
0,0,Elden Ring,25-Feb-22,"['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']",17K,3.8K,...,Bandai Namco Entertainment,2022,Feb,Game_type.AAA,Adventure,RPG,NaN,NaN,NaN,NaN
1,1,Hades,10-Dec-19,['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",21K,3.2K,...,NaN,2019,Dec,Game_type.INDIE,Adventure,Brawler,Indie,RPG,NaN,NaN
2,2,The Legend of Zelda: Breath of the Wild,3-Mar-17,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",30K,2.5K,...,Nintendo,2017,Mar,Game_type.AAA,Adventure,RPG,NaN,NaN,NaN,NaN
3,3,Undertale,15-Sep-15,"['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",28K,679,...,Toby Fox,2015,Sep,Game_type.INDIE,Adventure,Indie,RPG,Turn Based Strategy,NaN,NaN
4,4,Hollow Knight,24-Feb-17,['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",21K,2.4K,...,Team Cherry,2017,Feb,Game_type.INDIE,Adventure,Indie,Platform,NaN,NaN,NaN


In [3]:
# Implementing Overview+Detail using semantic zoom
year_selection = alt.selection_point(fields=["Year Released"])
base = alt.Chart(data).properties(width=800, height=250)

#Specify our overview chart
overview = alt.Chart(data).mark_bar().encode(
    x = alt.X("Year Released").scale(zero=False),
    y = alt.Y("mean(Rating)"),
    color = alt.condition(year_selection, alt.value("orange"), alt.value("lightgrey"))
).add_params(year_selection).properties(width=400, height=250)

# Specify our detail chart
detail = base.mark_bar().encode(
    y = "mean(Rating)",    
    x = "Title",
    color = alt.condition(alt.FieldEqualPredicate(field="Game Type", equal="Game_type.INDIE"), alt.value("blue"), alt.value("red")),
    tooltip=["Title", "Rating", "Year Released"]
).transform_filter(year_selection).properties(width=800, height=250)

overview | detail

alt.HConcatChart(...)

In [29]:
# Implement genre filtering
genre_list = ['Adventure', 'RPG', 'Indie', 'Turn Based Strategy', 'Platform', 'Simulator', 'Brawler', 'Puzzle', 'Shooter', 'Music', 'Strategy', 
              'Fighting', 'Visual Novel', 'Tactical', 'Racing', 'Arcade', 'Point-and-Click', 'Sport', 'Card & Board Game', 'Real Time Strategy']

genre_dropdown = alt.binding_select(options=genre_list, name="Select a genre to filter: ")

genre_selector = alt.param(name='SelectorName', value='Adventure', bind=genre_dropdown)

alt.Chart(data).mark_circle().encode(
    x = alt.X("Release Date:T"),
    y = "Rating",
    color = alt.condition(alt.FieldEqualPredicate(field="Game Type", equal="Game_type.INDIE"), alt.value("blue"), alt.value("red")),
    # size = "Plays", # don't implement size until we have more reliable data on sales
    tooltip=["Title", "Rating", "Year Released"],
    opacity=alt.condition(
        alt.expr.test(alt.expr.regexp(genre_selector, 'i'), alt.datum.Genres), 
        alt.value(1), 
        alt.value(.05))
).properties(width=800, height=500).add_params(genre_selector).interactive()

alt.Chart(...)

In [4]:
# Uses only Genre 1 as the 'main genre' for selection
genre_list = ['Adventure', 'RPG', 'Indie', 'Turn Based Strategy', 'Platform', 'Simulator', 'Brawler', 'Puzzle', 'Shooter', 'Music', 'Strategy', 
              'Fighting', 'Visual Novel', 'Tactical', 'Racing', 'Arcade', 'Point-and-Click', 'Sport', 'Card & Board Game', 'Real Time Strategy']
genre_dropdown = alt.binding_select(options=genre_list, name="Select a genre to filter: ")

selection = alt.selection_point(fields=["Genre 1"], bind=genre_dropdown)

color = alt.condition(
    selection,
    alt.Color('Genre 1:N', scale=alt.Scale(scheme='category20')),
    alt.value('lightgray')
)
alt.Chart(data).mark_circle().encode(
    x = alt.X("Release Date:T"),
    y = "Rating",
    color=color,
    # TODO: maybe add something to make size bigger if game is indie, or maybe change shape if that's an option?
    tooltip=["Title", "Rating", "Year Released"],
    opacity=alt.condition(selection,alt.value(1),alt.value(.1)),
).add_params(selection).properties(width=800, height=500).interactive()



alt.Chart(...)

In [5]:
# Comparing genres, Indie vs. AAA
dropdown = alt.binding_select(options=["Game_type.INDIE", "Game_type.AAA"], name="Select a type to filter: ")

selection = alt.selection_point(fields=["Game Type"], bind=dropdown)

color = alt.condition(
    selection,
    alt.Color('Genre 1:N', scale=alt.Scale(scheme='category20')),
    alt.value('lightgray')
)
alt.Chart(data).mark_circle().encode(
    x = alt.X("Release Date:T"),
    y = "Rating",
    color=color,
    tooltip=["Title", "Rating", "Year Released"],
    opacity=alt.condition(selection,alt.value(1),alt.value(.1)),
).add_params(selection).properties(width=800, height=500).interactive()



alt.Chart(...)